**PROJECT MAIN**

Load Images Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/food41")

print("Path to dataset files:", path)


100%|██████████| 5.30G/5.30G [00:55<00:00, 102MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kmader/food41/versions/5


In [ ]:
import os

data_path = 'drive/MyDrive/DSdata'

food_names = []

# Extract food names from folders
for food_name in os.listdir(data_path):
  full_path = os.path.join(data_path, food_name)
  if os.path.isdir(full_path):
    food_names.append(food_name)

print(food_names)

['archive (1)']


In [ ]:
# Downscale images (ChatGPT)
# Take our raw images
import os
from PIL import Image
from tqdm import tqdm

# Source and target directories
SRC_DIR = "images"        # original images folder with class subfolders
DST_DIR = "food-128"      # output folder

size = (128, 128)  # target resolution

# Create output root directory
os.makedirs(DST_DIR, exist_ok=True)

# Loop through every class folder
for class_name in os.listdir(SRC_DIR):
    src_class_path = os.path.join(SRC_DIR, class_name)
    dst_class_path = os.path.join(DST_DIR, class_name)

    # Skip non-directories (just in case)
    if not os.path.isdir(src_class_path):
        continue

    os.makedirs(dst_class_path, exist_ok=True)

    # Resize each image
    for img_name in tqdm(os.listdir(src_class_path), desc=f"Resizing {class_name[:20]}"):
        src_img_path = os.path.join(src_class_path, img_name)
        dst_img_path = os.path.join(dst_class_path, img_name)

        try:
            img = Image.open(src_img_path).convert("RGB")
            img = img.resize(size, Image.BILINEAR)
            img.save(dst_img_path, quality=90)
        except Exception as e:
            print(f"Skipped {src_img_path}: {e}")

print("All images resized to 128x128 and saved in food-128/")

Resizing beignets:  17%|█▋        | 166/1000 [00:02<00:12, 66.45it/s]